Следующие 2 ячейки выполнять не нужно. Они нужны лишь для работы в Google Colab

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cd /content/drive/My Drive/Colab Notebooks/data

/content/drive/My Drive/Colab Notebooks/data


Запускать можно код, начиная со следующей ячейки

In [1]:
import pandas as pd
import numpy as np

In [2]:
flags_descripts = ["country", "zone", "square", "population", "continent_by_square", "continent_by_population","language","religion",
                   "vertical_count","horizontal_count","colors_count","has_red","has_green","has_blue","has_gold","has_white",
                   "has_black","has_orange", "dominant_color"]

In [3]:
df_train = pd.read_csv('Flags.csv', names=flags_descripts, sep=";")
print (df_train.shape)

(194, 19)


In [4]:
print (df_train)

                  country  zone  square  population  continent_by_square  \
0             Afghanistan     1     648          16                    1   
1                 Albania     1      29           3                    5   
2                 Algeria     1    2388          20                    2   
3          American-Samoa     3       0           0                    6   
4                 Andorra     1       0           0                    5   
5                  Angola     2    1247           7                    2   
6                Anguilla     4       0           0                    3   
7         Antigua-Barbuda     4       0           0                    3   
8               Argentina     3    2777          28                    4   
9               Argentine     3    2777          28                    4   
10              Australia     2    7690          15                    6   
11                Austria     1      84           8                    5   
12          

In [5]:
name_scale = ["country", "zone", "language", "has_red", "has_green", "has_blue", "has_gold", "has_white", "has_black", "has_orange", "dominant_color"]
strong_scale = ["square", "population", "vertical_count", "horizontal_count","colors_count"]
order_scale = ["continent_by_square", "continent_by_population"]


In [6]:
def calculate_strong_scale_dists(np_strong_scale_array):
  
  np_strong_scale_array = np_strong_scale_array[:,np.newaxis]
  tmp_dists = np.zeros((np_strong_scale_array.shape[0], np_strong_scale_array.shape[0]), np.float)
  
  for i in range(np_strong_scale_array.shape[0]):
     tmp_dists[i] = np.sum(np.abs(np_strong_scale_array - np_strong_scale_array[i]) / np.abs(np.max(np_strong_scale_array) - np.min(np_strong_scale_array)), axis = 1)  

  return list(tmp_dists)

In [7]:
def calculate_name_scale_dists(np_name_scale_array):
  tmp_dists = np.zeros((np_name_scale_array.shape[0], np_name_scale_array.shape[0]), np.float)
  for i in range(np_name_scale_array.shape[0]):
    for j in range(np_name_scale_array.shape[0]):
      for k in range(np_name_scale_array.shape[0]):
        if (((np_name_scale_array[i] == np_name_scale_array[k]) and (np_name_scale_array[j] != np_name_scale_array[k]))
        or (np_name_scale_array[i] != np_name_scale_array[k]) and (np_name_scale_array[j] == np_name_scale_array[k])):
          tmp_dists += 1.0
  tmp_dists / np_name_scale_array.shape[0]
  return list(tmp_dists)

In [8]:
def calculate_order_scale_dists(np_order_scale_array):
  tmp_dists = np.zeros((np_order_scale_array.shape[0], np_order_scale_array.shape[0]), np.float)
  for i in range(np_order_scale_array.shape[0]):
    for j in range(np_order_scale_array.shape[0]):
      for k in range(np_order_scale_array.shape[0]):

        if (((np_order_scale_array[i] > np_order_scale_array[k]) and (np_order_scale_array[j] < np_order_scale_array[k]))
        or ((np_order_scale_array[i] < np_order_scale_array[k]) and (np_order_scale_array[j] > np_order_scale_array[k]))):
          tmp_dists += 1.0

        elif (((np_order_scale_array[i] != np_order_scale_array[k]) and (np_order_scale_array[j] == np_order_scale_array[k]))
        or ((np_order_scale_array[i] == np_order_scale_array[k]) and (np_order_scale_array[j] != np_order_scale_array[k]))):
          tmp_dists += 0.5
  tmp_dists / (np_order_scale_array.shape[0] - 1)
  return list(tmp_dists)

In [ ]:
name_scale_dists = []
for i in name_scale:
  name_scale_dists.append(calculate_name_scale_dists(np.array(df_train[i])))

order_scale_dists = []
for i in order_scale:
  order_scale_dists.append(calculate_order_scale_dists(df_train[i]))

strong_scale_dists = []
for i in strong_scale:
  strong_scale_dists.append(calculate_strong_scale_dists(np.array(df_train[i])))


In [ ]:
for i in range(len(strong_scale)):
  for j in range(df_train.shape[0]):
    for k in range(df_train.shape[0]):
      if (j == k):
        assert strong_scale_dists[i][j][k] == 0.0
      else:
        assert strong_scale_dists[i][j][k] == strong_scale_dists[i][k][j]



In [ ]:
dists = np.zeros((df_train.shape[0], df_train.shape[0]), np.float)

for i in range(len(name_scale)):
  np_name_scale_dists = np.array(name_scale_dists[i])
  np_name_scale_dists = np.power(np_name_scale_dists, 2)
  dists = np.sum(dists, np_name_scale_dists)

# for i in range(len(order_scale)):
#   np_order_scale_dists = np.array(order_scale_dists[i])
#   dists = np.sum(dists, np.power(np_order_scale_dists, 2))

# for i in range(len(order_scale)):
#   np_strong_scale_dists = np.array(nstrong_scale_dists[i])
#   dists = np.sum(dists, np.power(np_strong_scale_dists), 2)

# n = len(name_scale) + len(order_scale) + len(strong_scale)
# dists = 1/sqrt(n) * np.power(dists, 0.5)

TypeError: ignored